In [2]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

In [1]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup
from scraper import *

2024-05-24 22:40:01.093 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.096 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.098 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.099 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.100 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.102 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-24 22:40:01.104 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


# scraping of the performance info

In [2]:
from scraper import *
from utils import *

list_url = ['https://www.vlr.gg/318937/karmine-corp-vs-team-heretics-champions-tour-2024-emea-stage-1-w5']


In [5]:
def pick_and_ban_scraper(list_url):
    """
    Function that extract the picks and ban data from a vlr.gg url (page of a match between two teams)
    The economy data aims the economy display in the vlr.gg page and gather the data displayed in the tables in a textual form

    parameter:
        list_url : list of url representing the different matches
    
    return:
        result : a dataframe of all the extracted data flatten
    """

    match_stats = []

    for matchnum in range(len(list_url)):

        url = list_url[matchnum]

        unique_match_id = url.split('/')[3]

        source_match = requests.get(url=url).text
        soup_match = BeautifulSoup(source_match, features="html.parser")

        stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]

        series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()

        try:

            picks_bans = soup_match.findAll('div', {'class':'match-header-note'})[0].text.strip().split(";")

            headers_match = ["Id","Stage","Series","Team Name", "Bans", "Picks", "Decider"]
            df_match = pd.DataFrame(columns=headers_match)

            [pick_or_ban_team1, pick_or_ban_team2] = reorganize_phrases(picks_bans)

            row1 = [unique_match_id, stage, series] + pick_or_ban_team1
            row2 = [unique_match_id, stage, series]+ pick_or_ban_team2

            length = len(df_match)
            df_match.loc[length] = row1
            df_match.loc[length+1] = row2

            match_stats.append(df_match)   
        except NameError:
            print(f'Matches on the {series}, {stage} are not played yet...')
            print(NameError)
            continue


    try:
        result = pd.concat(match_stats).reset_index(drop=True)
        result = result.apply(pd.to_numeric, errors='ignore')
    except:
        result = None

    return result

In [6]:
t = pick_and_ban_scraper(list_url)

In [8]:
t

,Id,Stage,Series,Team Name,Bans,Picks,Decider
0,318937,Regular Season,Week 5,KC,"[Sunset, Ascent]",[Icebox],[Bind]
1,318937,Regular Season,Week 5,TH,"[Split, Breeze]",[Lotus],[Bind]


In [1]:
from scraper import *

main("https://www.vlr.gg/event/matches/1924/champions-tour-2024-pacific-kickoff/?series_id=all")

Get all the matches...
Fetching General Data...
Fetching Performance Data...
Fetching Economy Data...
Fetching Picks and Bans Data...
Saving the data...
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\general_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\performance_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\economy_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\pick_ban_data_champions-tour-2024-pacific-kickoff.csv


0

In [6]:
result.head()

,Stage,Series,Team Name,Bans,Picks,Decider
0,Group Stage,Decider (C),M8,"[Bind, Ascent]",[Breeze],[Sunset]
1,Group Stage,Decider (C),VIT,"[Icebox, Split]",[Lotus],[Sunset]
2,Play-Ins,Group B,GTZ,"[Lotus, Split, Haven]",[],[Pearl]
3,Play-Ins,Group B,MOUZ,"[Fracture, Ascent, Bind]",[],[Pearl]
